<a href="https://colab.research.google.com/github/mrudulagavas/All-About-AI-ML/blob/main/Transformer_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transformer from Scratch - "Attention Is All You Need"

In [1]:
import torch
print(torch.__version__)


2.6.0+cu124


In [ ]:
!pip install torch==2.2.0 torchtext==0.17.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.metrics import bleu_score
import spacy
import random

1. Load English and German tokenizers

In [4]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.11/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3.11/dist-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/usr/local/lib/python3.11/dist-packages/spacy/compat.py", line 4, in <module>
    from thinc.

In [5]:
spacy_de = spacy.load("de_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

In [6]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [7]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

Build vocab

In [8]:
SRC_TOKENIZER = tokenize_de
TGT_TOKENIZER = tokenize_en

In [9]:
def yield_tokens(data_iter, tokenizer):
    for src, tgt in data_iter:
        yield tokenizer(src)

In [10]:
def build_vocab(dataset, tokenizer):
    return build_vocab_from_iterator(yield_tokens(dataset, tokenizer), specials=["<unk>", "<pad>", "<bos>", "<eos>"]).set_default_index(0)

Load dataset and preprocess

In [11]:
!pip install portalocker>=2.0.0


In [12]:
!pip install -U torchdata


  Using cached torchdata-0.11.0-py3-none-any.whl.metadata (6.3 kB)
Using cached torchdata-0.11.0-py3-none-any.whl (61 kB)
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.7.1
    Uninstalling torchdata-0.7.1:
      Successfully uninstalled torchdata-0.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.17.0 requires torchdata==0.7.1, but you have torchdata 0.11.0 which is incompatible.


In [13]:
!pip install --upgrade torch==2.2.0 torchtext==0.17.0 torchdata==0.7.1


  Using cached torchdata-0.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached torchdata-0.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.11.0
    Uninstalling torchdata-0.11.0:
      Successfully uninstalled torchdata-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires torchdata==0.11.0, but you have torchdata 0.7.1 which is incompatible.


In [3]:
from torchtext.vocab import build_vocab_from_iterator

def build_vocab(dataset, tokenizer, specials=["<unk>", "<pad>", "<bos>", "<eos>"]):
    def yield_tokens(data_iter):
        for src, tgt in data_iter:
            yield tokenizer(src)  # For source
    vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=specials)
    vocab.set_default_index(vocab["<unk>"])
    return vocab



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [4]:
print("SRC_TOKENIZER defined?", 'SRC_TOKENIZER' in globals())
print("TGT_TOKENIZER defined?", 'TGT_TOKENIZER' in globals())


SRC_TOKENIZER defined? False
TGT_TOKENIZER defined? False


In [5]:
import spacy

# Load German and English spaCy models
spacy_de = spacy.load("de_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

# Define tokenizers
SRC_TOKENIZER = lambda text: [tok.text.lower() for tok in spacy_de.tokenizer(text)]
TGT_TOKENIZER = lambda text: [tok.text.lower() for tok in spacy_en.tokenizer(text)]


In [6]:
from torchtext.datasets import Multi30k
train_iter = Multi30k(split='train', language_pair=('de', 'en'))

# FIX: use list() instead of to_map_style_dataset
train_dataset = list(train_iter)

src_vocab = build_vocab(train_dataset, SRC_TOKENIZER)
tgt_vocab = build_vocab(train_dataset, TGT_TOKENIZER)

PAD_IDX = tgt_vocab["<pad>"]
BOS_IDX = tgt_vocab["<bos>"]
EOS_IDX = tgt_vocab["<eos>"]


Collate function for DataLoader

In [7]:
def collate_batch(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_tensor = torch.tensor([src_vocab[token] for token in SRC_TOKENIZER(src_sample)], dtype=torch.long)
        tgt_tensor = torch.tensor([tgt_vocab[token] for token in TGT_TOKENIZER(tgt_sample)], dtype=torch.long)
        src_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_tensor, torch.tensor([EOS_IDX])]))
        tgt_batch.append(torch.cat([torch.tensor([BOS_IDX]), tgt_tensor, torch.tensor([EOS_IDX])]))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch.transpose(0, 1), tgt_batch.transpose(0, 1)

In [8]:
from torch.utils.data import DataLoader


In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)

Model Architecture

In [10]:
import torch.nn as nn


In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

def scaled_dot_product_attention(q, k, v, mask=None):
    d_k = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    attn = torch.softmax(scores, dim=-1)
    return torch.matmul(attn, v), attn

In [12]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.linear_q = nn.Linear(d_model, d_model)
        self.linear_k = nn.Linear(d_model, d_model)
        self.linear_v = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        B = q.size(0)
        q = self.linear_q(q).view(B, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = self.linear_k(k).view(B, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = self.linear_v(v).view(B, -1, self.num_heads, self.d_k).transpose(1, 2)

        out, _ = scaled_dot_product_attention(q, k, v, mask)
        out = out.transpose(1, 2).contiguous().view(B, -1, self.num_heads * self.d_k)
        return self.fc_out(out)

In [13]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))

In [14]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        x = self.norm1(x + self.attn(x, x, x, mask))
        x = self.norm2(x + self.ff(x))
        return x

In [15]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff) for _ in range(N)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src, mask=None):
        x = self.embedding(src)
        x = self.pos_encoder(x)
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [16]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, x, enc_out, src_mask=None, tgt_mask=None):
        x = self.norm1(x + self.self_attn(x, x, x, tgt_mask))
        x = self.norm2(x + self.cross_attn(x, enc_out, enc_out, src_mask))
        x = self.norm3(x + self.ff(x))
        return x

In [17]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=512, N=6, heads=8, d_ff=2048):
        super().__init__()
        self.encoder = TransformerEncoder(src_vocab, d_model, N, heads, d_ff)
        self.decoder_embedding = nn.Embedding(tgt_vocab, d_model)
        self.pos_decoder = PositionalEncoding(d_model)
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, heads, d_ff) for _ in range(N)])
        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        enc_out = self.encoder(src, src_mask)
        x = self.decoder_embedding(tgt)
        x = self.pos_decoder(x)
        for layer in self.decoder_layers:
            x = layer(x, enc_out, src_mask, tgt_mask)
        return self.fc_out(x)

In [18]:
def generate_square_subsequent_mask(sz):
    mask = torch.triu(torch.ones((sz, sz)) == 1).transpose(0, 1)
    return mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

BLEU score evaluation

In [19]:
def translate(model, src_sentence, src_vocab, tgt_vocab, device):
    model.eval()
    src_tensor = torch.tensor([src_vocab[token] for token in SRC_TOKENIZER(src_sentence)], dtype=torch.long).unsqueeze(0).to(device)
    src_tensor = torch.cat([torch.tensor([[BOS_IDX]]).to(device), src_tensor, torch.tensor([[EOS_IDX]]).to(device)], dim=1)
    enc_src = model.encoder(src_tensor)
    tgt_indexes = [BOS_IDX]
    for i in range(50):
        tgt_tensor = torch.tensor(tgt_indexes, dtype=torch.long).unsqueeze(0).to(device)
        tgt_mask = generate_square_subsequent_mask(tgt_tensor.size(1)).to(device)
        output = model(src_tensor, tgt_tensor, None, tgt_mask)
        pred_token = output.argmax(2)[:, -1].item()
        tgt_indexes.append(pred_token)
        if pred_token == EOS_IDX:
            break
    return [tgt_vocab.get_itos()[i] for i in tgt_indexes[1:-1]]

In [20]:
def evaluate_bleu(model, data, num_samples=100):
    references = []
    hypotheses = []
    for idx, (src, tgt) in enumerate(data):
        if idx >= num_samples:
            break
        pred = translate(model, src, src_vocab, tgt_vocab, torch.device("cpu"))
        ref = [TGT_TOKENIZER(tgt)]
        references.append(ref)
        hypotheses.append(pred)
    return bleu_score(hypotheses, references)


In [21]:
import torch
import math

In [22]:
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [23]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    model = Transformer(len(src_vocab), len(tgt_vocab)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

    model.train()
    print("Training on one batch...")
    for src_batch, tgt_batch in train_dataloader:
        src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1)).to(device)
        output = model(src_batch, tgt_input, None, tgt_mask)

        output = output.reshape(-1, output.size(-1))
        tgt_output = tgt_output.reshape(-1)

        loss = criterion(output, tgt_output)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Loss after one batch: {loss.item():.4f}")
        break  # only 1 batch demo

Device: cuda
Training on one batch...
Loss after one batch: 9.8053


In [24]:
    sample_sentence = "Ein kleines Mädchen spielt draußen."
    print("\nTranslating:", sample_sentence)
    translation = translate(model, sample_sentence, src_vocab, tgt_vocab, device)
    print("Translation Output:", " ".join(translation))


Translating: Ein kleines Mädchen spielt draußen.
Translation Output: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


Modifying tokenize_de and tokenize_en

In [25]:
def tokenize_de(text):
    return [tok.text.lower() for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]


rebuilding vocab

In [26]:
src_vocab = build_vocab(train_dataset, SRC_TOKENIZER)
tgt_vocab = build_vocab(train_dataset, TGT_TOKENIZER)


training for more epoch

In [28]:
NUM_EPOCHS = 3
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    model.train()
    for src_batch, tgt_batch in train_dataloader:
        src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1)).to(device)
        output = model(src_batch, tgt_input, None, tgt_mask)

        output = output.reshape(-1, output.size(-1))
        tgt_output = tgt_output.reshape(-1)

        loss = criterion(output, tgt_output)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 2711.3199
Epoch 2, Loss: 2710.1922
Epoch 3, Loss: 2709.3586
